# Sign processing

This notebook starts by fetching observations from the WFS defined layer, the main idea is to convert the sps scripts to python code, which we could then execute directly via github and/or process without having to have SPSS installed.
The notebook requires `pandas`, as per the instructions in the requirements file.

In [1]:
from owslib.wfs import WebFeatureService
from datetime import datetime
import time
import json

import numpy as np
import pandas as pd
import pyproj
import folium
from pandas_geojson import to_geojson, write_geojson
from datetime import date

## Configuration
Configuration variables are defined here, this is only temporary since this code will all be converted to scripts.

In [2]:
url = "https://opendata.apps.mow.vlaanderen.be/opendata-geoserver/awv/wfs?version=2.0.0" #&service=wfs&request=GetCapabilities"
wfs = WebFeatureService(url=url, version="2.0.0", timeout=3600)
vb_type_name = "awv:Verkeersborden.Vlaanderen_Borden"

# Configuration
# Output file where we will store the WFS results
feature_output_file = "../python_output/feature_output.csv"
# Output where the geojson file will be stored
geojson_output_file = "../python_output/geojson_output.json"
# Previous processed data, used to filter out previous data
previous_processed_date = "31/07/2022"
# Previous traffic signs
traffic_signs_info = "../find-interesting-signs/road_signs_cleaned.csv"
# Traffic sign processing output file
processing_output_file = "maproulette.csv"


## Fetch number of features
Fetch all the features for the required layer from the WFS service, we use this later on to query for them all.

In [3]:
def get_total_features_by_type(feature_type):
    response = wfs.getfeature(typename=feature_type, outputFormat="json", maxfeatures=1)
    r = response.read()
    d = r.decode('UTF-8')
    j = json.loads(d)
    return j['totalFeatures'] 

total_features = get_total_features_by_type(vb_type_name)
print("{}: #features = {}".format(datetime.now(), total_features))

2022-11-06 20:13:40.858953: #features = 1525825


## Obtain and store the signs
Fetch data from WFS, remove line breaks and store into the defined csv file.

In [4]:
def remove_linebreaks(data):
    replace1 = data.replace(b'\n',b' ')
    replace2 = replace1.replace(b'\r ',b'\r\n')
    return replace2

def get_and_store_features(file_name, feature_type, max_features):
    response = wfs.getfeature(typename=feature_type, maxfeatures=max_features, outputFormat="csv", startindex=0)
    cleaned_response = remove_linebreaks(response.read())
    decoded_response = cleaned_response.decode('UTF-8')

    with open(file=file_name, encoding='UTF-8', mode='w', newline='') as csvfile:
        csvfile.write(decoded_response)
        
print("{}: Starting fetching data from WFS service".format(datetime.now()))
get_and_store_features(feature_output_file, vb_type_name, total_features)
print("{}: WFS data stored in {}".format(datetime.now(), feature_output_file))

2022-11-06 20:13:40.878478: Starting fetching data from WFS service
2022-11-06 20:31:32.295632: WFS data stored in ../python_output/feature_output.csv


## Process data

Load the signs data in `panda` dataframes, this data is filtered by the `previous_processed_date` and joined with the signs metadata by `bordcode`.

**Note:** All this code is dataset specific, ideally this should be abstracted away, including column definitions.

In [5]:
feature_df = pd.read_csv(feature_output_file)

In [6]:
feature_df.dtypes

FID                       object
UUID                      object
externe_id_bord           object
locatie_x                float64
locatie_y                float64
bordcode                  object
opstelhoogte               int64
breedte                  float64
hoogte                   float64
folietype                float64
leverancier               object
vorm                      object
fabricage_type            object
fabricage_jaar           float64
fabricage_maand          float64
besteknummer              object
opmerkingen               object
beheerder                 object
datum_plaatsing           object
parameters                object
bevestigingsProfielen      int64
beugels                    int64
type_bevestiging          object
id_opstelling              int64
id_aanzicht                int64
geometry                  object
dtype: object

### Date filtering

Filter the dataframe for all signs with date greater than the `previous_processed_date` configuration value. This is done by: 1) converting the `datum_plaatsing` to date in the `date` column, and 2) filtering the dataframe.

In [7]:
print(f"The file containes {len(feature_df)} features before filtering by date.")
feature_df['date'] = pd.to_datetime(feature_df['datum_plaatsing'], errors = 'coerce', infer_datetime_format=True)
filter_mask = feature_df['date'].notna() & (feature_df["date"] > previous_processed_date)
filtered_df = feature_df[filter_mask]
print(f"The file contains {len(filtered_df)} features after filtering by date greater than {previous_processed_date}.")

The file containes 1525825 features before filtering by date.
The file contains 3539 features after filtering by date greater than 31/07/2022.


/opt/anaconda3/envs/trafficsignproject/lib/python3.8/site-packages/pandas/core/arraylike.py:56: UserWarning: Parsing '31/07/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return self._cmp_method(other, operator.gt)


### Data parsing and conversion

Some small conversion on the `bordcode` field, as per the SPS code. This code also create the identifier removing the string from the `FID` value. Latitude and longitude are converted from [EPSG:31370](https://epsg.io/31370) to [EPSG:4326](https://epsg.io/4326) aka WGS84.

In [8]:
from pyproj import Transformer
transformer = Transformer.from_crs("epsg:31370", "epsg:4326", always_xy=True)

def convertCoords(row):
    # Transform columns based on locatie_x (longitude) and locatie_y (latitude).
    longitude ,latitude = transformer.transform(row['locatie_x'],row['locatie_y'])
    return pd.Series({'longitude': longitude,'latitude': latitude})

# convert coordinates
filtered_df[['longitude','latitude']] = filtered_df.apply(convertCoords,axis=1)
# Bordcode processing, remove Z from it and add (zone) description.
filtered_df['bordcode'] = filtered_df.apply(lambda row: (f"{row['bordcode'][1:]} (zone)" if row['bordcode'].startswith('Z') else row['bordcode']).replace("/", ""), axis=1)
# Replace strings from FID
filtered_df['id'] = filtered_df['FID'].str.replace('Verkeersborden.Vlaanderen_Borden.','')
filtered_df.drop(columns=['FID'])
# This will need require some cleaning on the parameters as well. Probably better to do it before saving.

/var/folders/p7/r1jmp1355rz7j2hr9bb1snz40000gn/T/ipykernel_7506/322692347.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['longitude','latitude']] = filtered_df.apply(convertCoords,axis=1)
/var/folders/p7/r1jmp1355rz7j2hr9bb1snz40000gn/T/ipykernel_7506/322692347.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['longitude','latitude']] = filtered_df.apply(convertCoords,axis=1)
/var/folders/p7/r1jmp1355rz7j2hr9bb1snz40000gn/T/ipykernel_7506/322692347.py:12: SettingWithCopyWa

,UUID,externe_id_bord,locatie_x,locatie_y,bordcode,opstelhoogte,breedte,hoogte,folietype,leverancier,...,bevestigingsProfielen,beugels,type_bevestiging,id_opstelling,id_aanzicht,geometry,date,longitude,latitude,id
2294,a289ab32-9ed1-4ca8-8bb3-22ceb146f694,NaN,189348.290,189995.200,C43,1500,700.0,700.0,NaN,NaN,...,2,0,Steun,1267160,1883330,POINT (189348.29 189995.2),2022-09-08,4.929533,51.018815,2875074
6151,e21afe04-7aec-4270-b14a-fb849a4c6a52,NaN,149472.594,199430.312,C9-C11,1000,400.0,400.0,NaN,NaN,...,1,0,Steun,1135073,1562263,POINT (149472.59375 199430.3125),2050-01-01,4.361226,51.104965,2336848
9600,3b674c0c-7564-4cd6-9f87-3069aed6a18a,NaN,189276.230,189901.330,C43,1500,700.0,700.0,NaN,NaN,...,2,0,Steun,1267161,1883331,POINT (189276.23 189901.33),2022-09-08,4.928496,51.017976,2875075
9649,93ab770f-febc-4938-8e0d-870e6ad5f0ab,NaN,159993.310,208030.210,C31a,2160,700.0,700.0,NaN,NaN,...,2,0,Steun,1267163,1883334,POINT (159993.31 208030.21),2022-09-08,4.511676,51.182179,2875081
9694,90a6abcd-8c9b-475b-84fe-d847b1d77dcb,NaN,159993.310,208030.210,GIV,1500,700.0,200.0,NaN,NaN,...,1,0,Steun,1267163,1883334,POINT (159993.31 208030.21),2022-09-08,4.511676,51.182179,2875083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512195,308930d9-14ad-42a2-b548-75d1c83398f6,NaN,63590.680,179832.425,B1,1800,700.0,700.0,NaN,NaN,...,2,0,Steun,1246413,1838274,POINT (63590.68 179832.425),2022-11-02,3.139764,50.922378,2797136
1516843,b890af79-3e2f-491c-861f-2d37f5481933,NaN,159539.530,205787.150,M3,1500,700.0,400.0,NaN,NaN,...,2,0,Steun,1267156,1883324,POINT (159539.53 205787.15),2022-09-08,4.505127,51.162025,2875066
1519579,ee79068f-9682-49fd-85e7-73a494f8398f,NaN,149488.312,199451.188,D1d,1000,700.0,700.0,NaN,NaN,...,2,0,Steun,1135068,1562231,POINT (149488.3125 199451.1875),2050-01-01,4.361450,51.105153,2336782
1520359,1ad9cb97-fe19-494d-827f-fdae65da3587,NaN,159539.530,205787.150,F45b,1930,400.0,600.0,NaN,NaN,...,2,0,Steun,1267156,1883324,POINT (159539.53 205787.15),2022-09-08,4.505127,51.162025,2875067


In [9]:
sign_metadata = pd.read_csv(traffic_signs_info, sep=";", encoding = "ISO-8859-1")
sign_metadata.dtypes

bordcode              object
name                  object
help                  object
wiki                  object
key                   object
value                 object
confusion_possible    object
opinion                int64
dtype: object

### Join and grouping

Merge the sign metadata with the current dataset based on the `bordcode` field. Then group by `id_aanzicht` to identified clustered signs. After that we get the required values and store them based on `processing_output_file` configuration value.

In [10]:
# Join both datasets by the bordcode
joined_df = filtered_df.join(sign_metadata.set_index("bordcode"), on='bordcode')
# Remove NaN parameters and name
joined_df[['parameters', 'name']] = joined_df[['parameters','name']].fillna('')
joined_df.dtypes
display(joined_df)

,FID,UUID,externe_id_bord,locatie_x,locatie_y,bordcode,opstelhoogte,breedte,hoogte,folietype,...,longitude,latitude,id,name,help,wiki,key,value,confusion_possible,opinion
2294,Verkeersborden.Vlaanderen_Borden.2875074,a289ab32-9ed1-4ca8-8bb3-22ceb146f694,NaN,189348.290,189995.200,C43,1500,700.0,700.0,NaN,...,4.929533,51.018815,2875074,Speed limit,"Choose from: 5, 10, 15, 20, 30, 40, 50, 60, 70...",Key:maxspeed,,,1,1.0
6151,Verkeersborden.Vlaanderen_Borden.2336848,e21afe04-7aec-4270-b14a-fb849a4c6a52,NaN,149472.594,199430.312,C9-C11,1000,400.0,400.0,NaN,...,4.361226,51.104965,2336848,,NaN,NaN,NaN,NaN,NaN,NaN
9600,Verkeersborden.Vlaanderen_Borden.2875075,3b674c0c-7564-4cd6-9f87-3069aed6a18a,NaN,189276.230,189901.330,C43,1500,700.0,700.0,NaN,...,4.928496,51.017976,2875075,Speed limit,"Choose from: 5, 10, 15, 20, 30, 40, 50, 60, 70...",Key:maxspeed,,,1,1.0
9649,Verkeersborden.Vlaanderen_Borden.2875081,93ab770f-febc-4938-8e0d-870e6ad5f0ab,NaN,159993.310,208030.210,C31a,2160,700.0,700.0,NaN,...,4.511676,51.182179,2875081,,NaN,NaN,NaN,NaN,NaN,NaN
9694,Verkeersborden.Vlaanderen_Borden.2875083,90a6abcd-8c9b-475b-84fe-d847b1d77dcb,NaN,159993.310,208030.210,GIV,1500,700.0,200.0,NaN,...,4.511676,51.182179,2875083,,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512195,Verkeersborden.Vlaanderen_Borden.2797136,308930d9-14ad-42a2-b548-75d1c83398f6,NaN,63590.680,179832.425,B1,1800,700.0,700.0,NaN,...,3.139764,50.922378,2797136,Give way,,Key:give_way,,,1,1.0
1516843,Verkeersborden.Vlaanderen_Borden.2875066,b890af79-3e2f-491c-861f-2d37f5481933,NaN,159539.530,205787.150,M3,1500,700.0,400.0,NaN,...,4.505127,51.162025,2875066,Except for bicycles and mopeds,Check access and ONEWAY tags!!! Remove the one...,,oneway:mofa,no,,1.0
1519579,Verkeersborden.Vlaanderen_Borden.2336782,ee79068f-9682-49fd-85e7-73a494f8398f,NaN,149488.312,199451.188,D1d,1000,700.0,700.0,NaN,...,4.361450,51.105153,2336782,,NaN,NaN,NaN,NaN,NaN,NaN
1520359,Verkeersborden.Vlaanderen_Borden.2875067,1ad9cb97-fe19-494d-827f-fdae65da3587,NaN,159539.530,205787.150,F45b,1930,400.0,600.0,NaN,...,4.505127,51.162025,2875067,"No exit, except for bicycle","If you add a no_exit tag, it should be added t...",Key:no_exit,no_exit:bicycle,no,,1.0


In [11]:
grouped_df = joined_df.groupby('id_aanzicht', as_index=False).agg({
     'opinion': 'max', 
     'bordcode': ' | '.join,
     'latitude': 'max',
     'longitude': 'max',
     'parameters': lambda x : '|'.join(y for y in x if y != ''),
     'name': lambda x : '|'.join(y for y in x if y != ''),
     'datum_plaatsing': 'max',
     'id': 'max'})
grouped_df = grouped_df[grouped_df['opinion'] > 0]
print(f"Found {len(grouped_df)} signs after grouping by id_aanzicht")
display(grouped_df)

Found 916 signs after grouping by id_aanzicht


,id_aanzicht,opinion,bordcode,latitude,longitude,parameters,name,datum_plaatsing,id
0,281271,1.0,F49,50.813252,3.493733,,Pedestrian crossing,06/04/2046,407891
3,1562263,1.0,C9-C11 | B9 | C19 | C43,51.104965,4.361226,"Neen,50",Priority road|Pedestrians prohibited|Speed limit,01/01/2050,2336848
4,1562264,1.0,B9 | A1d | GIa | C43,51.104724,4.360795,"m,50|Neen,50",Priority road|Right curve followed by left cur...,01/01/2050,2336852
5,1562267,1.0,C9-C11 | C19,51.104743,4.360858,,Pedestrians prohibited,01/01/2050,2336855
6,1562270,1.0,B1,51.104746,4.360998,,Give way,01/01/2050,2336858
...,...,...,...,...,...,...,...,...,...
2371,1944299,1.0,B1,50.873029,3.439887,,Give way,11/10/2022,2990300
2372,1944300,1.0,B1,50.873392,3.439863,,Give way,11/10/2022,2990301
2376,1944889,1.0,C43 (zone) | B1,50.942938,3.157039,"neen,C43,[""400"",""Ja"",""50""],",Give way,11/02/2022,2991352
2379,1945327,1.0,B1,50.894338,3.735041,,Give way,10/02/2022,2992069


In [12]:
result = grouped_df.rename(columns = {
    "bordcode": "traffic_sign_code", 
    "parameters": "extra_text",
    "datum_plaatsing": "date_installed",
    "name": "traffic_sign_description"
})[['id', 'traffic_sign_code', 'extra_text', 'traffic_sign_description', 'date_installed', 'longitude', 'latitude']]
display(result)

,id,traffic_sign_code,extra_text,traffic_sign_description,date_installed,longitude,latitude
0,407891,F49,,Pedestrian crossing,06/04/2046,3.493733,50.813252
3,2336848,C9-C11 | B9 | C19 | C43,"Neen,50",Priority road|Pedestrians prohibited|Speed limit,01/01/2050,4.361226,51.104965
4,2336852,B9 | A1d | GIa | C43,"m,50|Neen,50",Priority road|Right curve followed by left cur...,01/01/2050,4.360795,51.104724
5,2336855,C9-C11 | C19,,Pedestrians prohibited,01/01/2050,4.360858,51.104743
6,2336858,B1,,Give way,01/01/2050,4.360998,51.104746
...,...,...,...,...,...,...,...
2371,2990300,B1,,Give way,11/10/2022,3.439887,50.873029
2372,2990301,B1,,Give way,11/10/2022,3.439863,50.873392
2376,2991352,C43 (zone) | B1,"neen,C43,[""400"",""Ja"",""50""],",Give way,11/02/2022,3.157039,50.942938
2379,2992069,B1,,Give way,10/02/2022,3.735041,50.894338


# Store results
Store the processing results in geojson format using `pandas_geojson`

In [13]:
geo_json = to_geojson(df=result, lat='latitude', lon='longitude',
                 properties=['id','traffic_sign_code','extra_text','traffic_sign_description', 'date_installed' ])
write_geojson(geo_json, filename=geojson_output_file)

## Visualize results
Simple visualization of the geojson results in folium, no custom popup for the time being.

In [14]:
folium_map = folium.Map(
    location=[50.8476, 4.3572],
    zoom_start=8,
)
folium.GeoJson(data=geo_json).add_to(folium_map)
folium_map